In [1]:
from lstm_architecture import one_hot, run_with_config
from sliding_window import sliding_window
import numpy as np
import cPickle as cp
import time

# Neural net's config.

In [2]:
class Config(object):
    """
    define a class to store parameters,
    the input should be feature mat of training and testing
    """

    def __init__(self, X_train, X_test):
        # Data shaping
        self.train_count = len(X_train)  # nb of training series
        self.test_data_count = len(X_test)  # nb of testing series
        self.n_steps = len(X_train[0])  # nb of time_steps per series
        self.n_classes = 18  # Final output classes, one classification per series

        # Training
        self.learning_rate = 0.001
        self.lambda_loss_amount = 0.005
        self.training_epochs = 100
        self.batch_size = 100
        self.clip_gradients = 15.0
        self.gradient_noise_scale = None
        self.keep_prob_for_dropout = 0.85  # **(1/3.0)  # Dropout is added on inputs and after each stacked layers (but not between residual layers).

        # Linear+relu structure
        self.bias_mean = 0.3
        self.weights_stddev = 0.2  # I would recommend between 0.1 and 1.0 or to change and use a xavier initializer

        ########
        # NOTE: I think that if any of the below parameters are changed,
        # the best is to readjust every parameters in the "Training" section
        # above to properly compare the architectures only once optimised.
        ########

        # LSTM structure
        self.n_inputs = len(X_train[0][0])  # Features count
        self.n_hidden = 28  # nb of neurons inside the neural network
        self.use_bidirectionnal_cells = True  # Use bidir in every LSTM cell, or not:

        # High-level deep architecture
        self.also_add_dropout_between_stacked_cells = False  # True

# Dataset-specific constants 

In [3]:
# Hardcoded number of sensor channels employed in the OPPORTUNITY challenge
NB_SENSOR_CHANNELS = 113
NB_SENSOR_CHANNELS_WITH_FILTERING = 149

# Hardcoded number of classes in the gesture recognition problem
NUM_CLASSES = 18

# Hardcoded length of the sliding window mechanism employed to segment the data
SLIDING_WINDOW_LENGTH = 24

# Length of the input sequence after convolutional operations
FINAL_SEQUENCE_LENGTH = 8

# Hardcoded step of the sliding window mechanism employed to segment the data
SLIDING_WINDOW_STEP = int(SLIDING_WINDOW_LENGTH/2)
SLIDING_WINDOW_STEP_SHORT = SLIDING_WINDOW_STEP

# Batch Size
BATCH_SIZE = 100

# Number filters convolutional layers
NUM_FILTERS = 64

# Size filters convolutional layers
FILTER_SIZE = 5

# Number of unit in the long short-term recurrent layers
NUM_UNITS_LSTM = 128

# load dataset and set sliding window

In [4]:
def load_dataset(filename):

    f = file(filename, 'rb')
    data = cp.load(f)
    f.close()

    X_train, y_train = data[0]
    X_test, y_test = data[1]

    print(" ..from file {}".format(filename))
    print(" ..reading instances: train {0}, test {1}".format(X_train.shape, X_test.shape))

    X_train = X_train.astype(np.float32)
    X_test = X_test.astype(np.float32)

    # The targets are casted to int8 for GPU compatibility.
    y_train = y_train.astype(np.uint8)
    y_test = y_test.astype(np.uint8)

    return X_train, y_train, X_test, y_test

print("Loading data...")
X_train, y_train, X_test, y_test = load_dataset('data/oppChallenge_gestures.data')

assert (NB_SENSOR_CHANNELS_WITH_FILTERING == \
        X_train.shape[1] or NB_SENSOR_CHANNELS == X_train.shape[1])

def opp_sliding_window(data_x, data_y, ws, ss):
    data_x = sliding_window(data_x,(ws,data_x.shape[1]),(ss,1))
    data_y = np.asarray([[i[-1]] for i in sliding_window(data_y,ws,ss)])
    data_x, data_y = data_x.astype(np.float32), \
    one_hot(data_y.reshape(len(data_y)).astype(np.uint8))
    print("after sliding window (testing): inputs {0}, targets {1}"\
          .format(X_test.shape, y_test.shape))
    return data_x, data_y


#--------------------------------------------
# Loading dataset
#--------------------------------------------


# Sensor data is segmented using a sliding window mechanism
X_test, y_test = opp_sliding_window(X_test, y_test, 
                        SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP_SHORT)
X_train, y_train = opp_sliding_window(X_train, y_train, 
                            SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)

for mat in [X_train, y_train, X_test, y_test]:
    print mat.shape

Loading data...
 ..from file data/oppChallenge_gestures.data
 ..reading instances: train (557963, 113), test (118750, 113)
after sliding window (testing): inputs (118750, 113), targets (118750,)
after sliding window (testing): inputs (9894, 24, 113), targets (9894, 18)
(46495, 24, 113)
(46495, 18)
(9894, 24, 113)
(9894, 18)


# Training

In [5]:
n_layers_in_highway = 3
n_stacked_layers = 3
trial_name = "{}x{}".format(n_layers_in_highway, n_stacked_layers)

for learning_rate in [0.001]:
    for lambda_loss_amount in [0.005]:
        print "learning_rate: {}".format(learning_rate)
        print "lambda_loss_amount: {}".format(lambda_loss_amount)
        print ""

        class EditedConfig(Config):
            def __init__(self, X, Y):
                super(EditedConfig, self).__init__(X, Y)

                # Edit only some parameters:
                self.learning_rate = learning_rate
                self.lambda_loss_amount = lambda_loss_amount
                # Architecture params:
                self.n_layers_in_highway = n_layers_in_highway
                self.n_stacked_layers = n_stacked_layers

        accuracy_out, best_accuracy, f1_score_out, best_f1_score = \
                      run_with_config(EditedConfig, X_train, y_train, X_test, y_test)
        print (accuracy_out, best_accuracy, f1_score_out, best_f1_score)

        with open('{}_result_opportunity_18.txt'.format(trial_name),'a') as f:
            
            f.write(str(learning_rate)+' \t'+str(lambda_loss_amount)+' \t'+ \
                    str(accuracy_out)+' \t'+str(best_accuracy)+' \t'+\
                    str(f1_score_out)+' \t'+str(best_f1_score)+'\n\n' )

        print "___________________"
    print ""
print "Done."

learning_rate: 0.001
lambda_loss_amount: 0.005

Some useful info to get an insight on dataset's shape and normalisation:
features shape, labels shape, each features mean, each features standard deviation
((9894, 24, 113), (9894, 18), 1.7156952e-05, 0.50001287)
the dataset is therefore properly normalised, as expected.
(24, ?, 113)
(?, 113)
(24, '(?, 113)')

Creating hidden #1:
bidir:
(24, '(?, 14)')
bidir:
(24, '(?, 14)')
bidir:
(24, '(?, 14)')
(24, '(?, 28)')

Creating hidden #2:
bidir:
(24, '(?, 14)')
bidir:
(24, '(?, 14)')
bidir:
(24, '(?, 14)')
(24, '(?, 28)')

Creating hidden #3:
bidir:
(24, '(?, 14)')
bidir:
(24, '(?, 14)')
bidir:
(24, '(?, 14)')
(24, '(?, 28)')

Unregularised variables:
LSTM_network/layer_1/pass_forward/relu_fc_biases_noreg:0
LSTM_network/layer_1/pass_backward/relu_fc_biases_noreg:0
LSTM_network/layer_1/LSTM_residual_0/pass_forward/relu_fc_biases_noreg:0
LSTM_network/layer_1/LSTM_residual_0/pass_backward/relu_fc_biases_noreg:0
LSTM_network/layer_1/LSTM_residual_

/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


iter: 0, train loss: 1.26639485359, train accuracy: 0.769999980927, train F1-score: 0.84154047619, test loss: 1.14398610592, test accuracy: 0.849302411079, test F1-score: 0.819828764792
iter: 1, train loss: 0.913951992989, train accuracy: 0.849999904633, train F1-score: 0.854181625702, test loss: 0.908376812935, test accuracy: 0.845057368279, test F1-score: 0.842529909531
iter: 2, train loss: 0.727368712425, train accuracy: 0.839999973774, train F1-score: 0.845345933264, test loss: 0.724536538124, test accuracy: 0.88023018837, test F1-score: 0.869155887342
iter: 3, train loss: 0.694170475006, train accuracy: 0.859999895096, train F1-score: 0.822911371237, test loss: 0.720071196556, test accuracy: 0.863351345062, test F1-score: 0.862657299902
iter: 4, train loss: 0.587053537369, train accuracy: 0.900000035763, train F1-score: 0.867452380952, test loss: 0.682242155075, test accuracy: 0.870830595493, test F1-score: 0.868958286763
iter: 5, train loss: 0.550367534161, train accuracy: 0.8600

iter: 44, train loss: 0.331044316292, train accuracy: 0.930000007153, train F1-score: 0.902623809524, test loss: 0.516022026539, test accuracy: 0.887204110622, test F1-score: 0.889722240372
iter: 45, train loss: 0.349649071693, train accuracy: 0.920000076294, train F1-score: 0.944095238095, test loss: 0.524660348892, test accuracy: 0.886193454266, test F1-score: 0.882473954971
iter: 46, train loss: 0.315815895796, train accuracy: 0.910000026226, train F1-score: 0.860568627451, test loss: 0.486249625683, test accuracy: 0.895896315575, test F1-score: 0.894202337284
iter: 47, train loss: 0.233751118183, train accuracy: 0.960000038147, train F1-score: 0.911182072829, test loss: 0.50943249464, test accuracy: 0.88922560215, test F1-score: 0.888510459391
iter: 48, train loss: 0.305129051208, train accuracy: 0.930000066757, train F1-score: 0.940606060606, test loss: 0.51904296875, test accuracy: 0.888113796711, test F1-score: 0.890891754451
iter: 49, train loss: 0.26487866044, train accuracy: 

iter: 88, train loss: 0.278152525425, train accuracy: 0.939999997616, train F1-score: 0.908290526316, test loss: 0.560994982719, test accuracy: 0.883565545082, test F1-score: 0.882805538387
iter: 89, train loss: 0.239042714238, train accuracy: 0.939999997616, train F1-score: 0.969111111111, test loss: 0.539392650127, test accuracy: 0.887305140495, test F1-score: 0.889747065272
iter: 90, train loss: 0.26443874836, train accuracy: 0.960000038147, train F1-score: 0.891424242424, test loss: 0.569280743599, test accuracy: 0.888821363449, test F1-score: 0.883503045043
iter: 91, train loss: 0.273909270763, train accuracy: 0.950000047684, train F1-score: 0.928393341076, test loss: 0.5293597579, test accuracy: 0.889831960201, test F1-score: 0.888490449528
iter: 92, train loss: 0.300491660833, train accuracy: 0.94000005722, train F1-score: 0.922977777778, test loss: 0.560269773006, test accuracy: 0.877096951008, test F1-score: 0.874528974264
iter: 93, train loss: 0.28093034029, train accuracy: 0